In [3]:
# Find the base for various number of mixture signals. Add the alternative base features sequentially.
# Select them according to the approximation to the external error.

import numpy as np
import matplotlib.pyplot as plt
from functions import *
from dataload import (load_data, get_clusters, gen_base, next_sample_dset)
from experiment import FeatureSelection
from plots import plt_lines

In [4]:

# Load data and data basis
iqdata, iqnoise = load_data()
dbasis = get_clusters()

# Create a data generator and the main class instance
# The indices of base centroids in iqdata
idx_Abase = list(dbasis.keys())
Abase = iqdata[idx_Abase].transpose()

# How to use the data:
# Generate new dset with randomly mixed signals as y plus a noise of desired level
class_sizes = [3, 3, 3, 3, 3, 3]  # Set sample size for each collided groups
# dset = gen_base(iqdata, iqnoise, dbasis, cls_sizes)
# print(list(dset[0].keys())) # The keys in each data sample

# This instance delivers a new y with answers
# get_next_sample = next_sample_dset(dset, idx_Abase) # Use after generation new dset
# This instance carry models for each y
# fs = FeatureSelection(Abase) # Use after get a new y

In [5]:
# The parameters of the computational experiment
# n_models = 6`
# n_classes = len(class_sizes)
# Later we check the reconstructed signal with one of the cluster's signals
# Check the reconstruction quality
n_noise_levels = 10 # r, rows
n_classes = len(class_sizes) # c, columns
noise_levels = np.linspace(0, 1, n_noise_levels)
cnt_resolvedA = np.zeros((n_noise_levels, n_classes))
cnt_resolved1 = cnt_resolvedA.copy()
cnt_resolved2 = cnt_resolvedA.copy()

n_samples = 3
for noise_level, r in zip(noise_levels, range(n_noise_levels)):
    for c in range(1,n_classes):

        class_sizes =  np.zeros(len(class_sizes), dtype=int)
        class_sizes[c] = n_samples
        # Generate the new noisy dset of mixtures and create a data sample generator
        # dset = get_dset(class_sizes, noise_level)
        dset = gen_base(iqdata, iqnoise, dbasis, cls_sizes=class_sizes, noise_level=noise_level)
        get_next_sample = next_sample_dset(dset, idx_Abase)

        i_resolvedA, i_resolved1, i_resolved2 = 0, 0, 0
        for i in range(n_samples):
            answer_y, answer_A, _, _, _ = next(get_next_sample)
            fs = FeatureSelection(Abase, max_models = 6) # Reset the list of models
            fs.MAX_BASIS = c
            fs.run(answer_y)
            best_A = fs.best_model()['fea']
            error = fs.best_model()['err']
            if set(answer_A) == set(best_A):
                i_resolvedA += 1
            len_resolved = len(set(answer_A) & set(best_A))
            if len_resolved > 1:
                i_resolved1 += 1
            if len_resolved > 2:
                i_resolved2 += 1

            print('rc', r,c, 'i', i, '12A:', i_resolved1, i_resolved2, i_resolvedA,  'answer:', answer_A, 'model:', best_A, 'err:', f'{ error:.2f}')
        cnt_resolvedA[r,c] = i_resolvedA
        cnt_resolved1[r,c] = i_resolved1
        cnt_resolved2[r,c] = i_resolved2

_ = 100 * cnt_resolved1[:, 1:] / n_samples
plt_lines(_, noise_levels, 'one')
_ = 100 * cnt_resolved2[:, 1:] / n_samples
plt_lines(_, noise_levels, 'two')
_ = 100 * cnt_resolvedA[:, 1:] / n_samples
plt_lines(_, noise_levels, 'all')

# np.save('../tmp/cnt_resolved1.npy', cnt_resolved1)
# np.save('../tmp/cnt_resolved2.npy', cnt_resolved2)
# np.save('../tmp/cnt_resolvedA.npy', cnt_resolvedA)

rc 0 1 i 0 12A: 0 0 1 answer: [48] model: [48] err: 0.24
rc 0 1 i 1 12A: 0 0 2 answer: [18] model: [18] err: 0.00
rc 0 1 i 2 12A: 0 0 3 answer: [52] model: [52] err: 0.54
rc 0 2 i 0 12A: 1 0 1 answer: [61, 6] model: [61, 6] err: 0.06
rc 0 2 i 1 12A: 1 0 1 answer: [14, 63] model: [63, 41] err: 3.44
rc 0 2 i 2 12A: 2 0 2 answer: [22, 63] model: [63, 22] err: 0.26
rc 0 3 i 0 12A: 1 0 0 answer: [31, 0, 60] model: [25, 31, 60] err: 0.56
rc 0 3 i 1 12A: 2 0 0 answer: [38, 39, 37] model: [4, 38, 39] err: 4.63
rc 0 3 i 2 12A: 2 0 0 answer: [6, 14, 62] model: [1, 12, 14] err: 4.42
rc 0 4 i 0 12A: 0 0 0 answer: [56, 7, 41, 44] model: [23, 62, 9, 44] err: 0.55
rc 0 4 i 1 12A: 1 1 1 answer: [57, 45, 5, 27] model: [5, 57, 27, 45] err: 0.91
rc 0 4 i 2 12A: 1 1 1 answer: [44, 32, 23, 40] model: [56, 32, 8, 57] err: 0.64
rc 0 5 i 0 12A: 0 0 0 answer: [35, 47, 16, 0, 27] model: [15, 60, 61, 20, 3] err: 0.50
rc 0 5 i 1 12A: 0 0 0 answer: [47, 6, 43, 59, 58] model: [21, 51, 37, 25, 31] err: 1.10
rc 0 5 i